In this notebook I aim to try different feature selection method, "farming" the built in features from rdkit

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import MACCSkeys


# getting chemical descriptors
chemDescr = pd.read_csv("./drugCombDBdata/descriptions/drug_chemical_info.csv")

#removing empty values
print(str(len(chemDescr[chemDescr["smilesString"]=="none"]))+" drugs with 'none' as smile")
print(str(len(chemDescr[pd.isna(chemDescr["smilesString"])]))+" drugs with NaN as smile")
print("     dropping both for now")


chemDescr = chemDescr[chemDescr["smilesString"]!="none"]
chemDescr = chemDescr[~pd.isna(chemDescr["smilesString"])]
chemDescr.head(2)

30 drugs with 'none' as smile
454 drugs with NaN as smile
     dropping both for now


,drugName,cIds,drugNameOfficial,molecularWeight,smilesString
0,Bendamustine,CIDs00065628,bendamustine,358.26284,CN1C2=C(C=C(C=C2)N(CCCl)CCCl)N=C1CCCC(=O)O
1,Lonidamine,CIDs00039562,lonidamine,321.15810,C1=CC=C2C(=C1)C(=NN2CC3=C(C=C(C=C3)Cl)Cl)C(=O)O


At first I want to get all the interesting chemical features of these drugs across the 3 rdkit levels. Then I'll try feature selection, last i'll look for representation of the cell line<br> <br>
As index I will use cIds for now

In [2]:
%%capture --no-display
molDf = chemDescr[["cIds","smilesString"]]
molDf.loc[:,'mols'] = [Chem.MolFromSmiles(mol) for mol in molDf.loc[:,"smilesString"]]
molDf.drop(["smilesString"],axis=1,inplace=True)
molDf.reset_index(inplace=True)
molDf.drop("index",axis=1,inplace=True)
molDf.head(2)

,cIds,mols
0,CIDs00065628,<rdkit.Chem.rdchem.Mol object at 0x7d4508fde300>
1,CIDs00039562,<rdkit.Chem.rdchem.Mol object at 0x7d44c9f614e0>


I'll start with the 2D descriptors:

To calculate all 2D Descriptors, I have to instantiate a Calculator object and feed it the names of used Descriptors<br>
it seems like the function works with Chem.Lipinski descriptors as well as with simple chem descriptors

In [205]:
# 777 is returned when the computation of something is not possible
molDescCalc2 = MolecularDescriptorCalculator(["NumAmideBonds"])
molDescCalc2.CalcDescriptors(mol=molDf.iloc[10,1])

(777,)

In [3]:
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

lipinskiDescriptors = ["FractionCSP3","HeavyAtomCount","NHOHCount","NOCount","NumAliphaticCarbocycles","NumAliphaticHeterocycles",
"NumAromaticCarbocycles","NumAromaticHeterocycles","NumHAcceptors","NumHDonors",
"NumHeteroatoms","NumRotatableBonds","NumSaturatedCarbocycles","NumSaturatedHeterocycles","RingCount"]

rdkitChemDescriptors = ["BalabanJ","BertzCT","Ipc","HallKierAlpha","Kappa1","Kappa2","Kappa3",
"Chi0","Chi1","Chi0n","Chi1n","Chi2n","Chi3n","Chi4n","Chi0v","Chi1v","Chi2v","Chi3v","Chi4v",
"MolLogP","MolMR","MolWt","HeavyAtomMolWt","NumValenceElectrons",
"NumAromaticRings","NumSaturatedRings","NumAliphaticRings","FractionCSP3","TPSA",
"LabuteASA"]

# I don't know about them and there are 57 of them. To see if they pop up as interesting I include 15 for now
MOEtypeDescriptors = ["PEOE_VSA1","PEOE_VSA2","PEOE_VSA3",
"SMR_VSA1","SMR_VSA2","SMR_VSA3",
"SlogP_VSA1","SlogP_VSA2","SlogP_VSA3",
"EState_VSA1","EState_VSA2","EState_VSA3",
"VSA_EState1","VSA_EState2","VSA_EState3"]

# descriptors not in: [Phi, NumAmideBonds, NumSpiroAtoms, NumBridgeheadAtoms, MQNs]
all2Ddescriptors = lipinskiDescriptors + rdkitChemDescriptors + MOEtypeDescriptors

molDescCalc = MolecularDescriptorCalculator(all2Ddescriptors)

# example mol
molDescCalc.CalcDescriptors(mol=molDf.iloc[10,1])
1

1

Compute all Descriptors for all drugs:

In [4]:
molsNfeatures = pd.DataFrame([molDescCalc.CalcDescriptors(mol) for mol in molDf.iloc[:,1]],columns=all2Ddescriptors)
molsNfeatures.set_index(molDf["cIds"], inplace=True)
molsNfeatures.head(2)
#name rows and cols - ~25 seconds

,FractionCSP3,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAromaticCarbocycles,NumAromaticHeterocycles,NumHAcceptors,NumHDonors,...,SMR_VSA3,SlogP_VSA1,SlogP_VSA2,SlogP_VSA3,EState_VSA1,EState_VSA2,EState_VSA3,VSA_EState1,VSA_EState2,VSA_EState3
cIds,,,,,,,,,,,,,,,,,,,,,
CIDs00065628,0.500000,23,1,5,0,0,1,1,4,1,...,9.551078,4.89991,45.476431,18.263031,5.969305,6.420822,24.601650,2.022224,17.421914,8.744802
CIDs00039562,0.066667,21,1,4,0,0,2,1,3,1,...,9.780485,0.00000,20.856317,6.544756,5.969305,5.693928,21.976247,1.632842,11.272932,15.096254


In [106]:
# PBF is non reproducible
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem

molEmbed = Chem.AddHs(molDf.iloc[1,1])
AllChem.EmbedMolecule(molEmbed)
rdMolDescriptors.CalcPBF(molEmbed)

0.9319947309097566

In [9]:
%%capture --no-display
# get Molembeddings 4 mins runtime OUTPUT SAVED AS PICKLE

# from rdkit.Chem import AllChem

# molDf["molEmbeds"] = pd.DataFrame([Chem.AddHs(mol) for mol in molDf.iloc[:,1]])

# for i in range(len(molDf["molEmbeds"])):
#     AllChem.EmbedMolecule(molDf.loc[i,"molEmbeds"])

save file to prevent from having to compute it all the time

In [14]:
# import _pickle as pickle

# with open('drug-embeddings.pkl', 'wb') as outp:
#     pickle.dump(molDf, outp)

In [5]:
import _pickle as pickle

with open('drug-embeddings.pkl', 'rb') as inp:
    molDf = pickle.load(inp)

In [6]:
%%capture --no-display
from rdkit.Chem.Descriptors3D import *

# properties I want to find
funcLst = [Eccentricity,InertialShapeFactor,NPR1,NPR2,PMI1,PMI2,PMI3,RadiusOfGyration,SpherocityIndex]
strLst = ["Eccentricity","InertialShapeFactor","NPR1","NPR2","PMI1","PMI2","PMI3","RadiusOfGyration","SpherocityIndex"]


descr3Ds = pd.DataFrame(molDf["cIds"])
descr3Ds.reset_index(inplace=True)

for j in range(len(funcLst)):
    tempLst = []
    for i in range(len(molDf.iloc[:,2])):
        try:
            tempLst.append(funcLst[j](molDf.iloc[i,2]))
        except:
            tempLst.append(None)
    descr3Ds[strLst[j]] = tempLst
descr3Ds.head(2)

,index,cIds,Eccentricity,InertialShapeFactor,NPR1,NPR2,PMI1,PMI2,PMI3,RadiusOfGyration,SpherocityIndex
0,0,CIDs00065628,0.993094,0.000967,0.117318,0.929722,960.955541,7615.377756,8191.025810,4.837408,0.107492
1,1,CIDs00039562,0.971951,0.000981,0.235183,0.912555,930.166315,3609.231638,3955.082953,3.636559,0.280885


In [104]:
# COMPARE HERE: not all 3D properties are persistent
descr3Ds.head(2)

,index,cIds,Eccentricity,InertialShapeFactor,NPR1,NPR2,PMI1,PMI2,PMI3,RadiusOfGyration,SpherocityIndex
0,0,CIDs00065628,0.984161,0.000802,0.177277,0.951306,1186.452924,6366.761478,6692.651244,4.45887,0.254355
1,1,CIDs00039562,0.949926,0.000679,0.312475,0.832344,1226.288560,3266.481660,3924.435649,3.61998,0.239146


In [7]:
allFeats = descr3Ds.join(molsNfeatures,on="cIds",how="left").drop_duplicates()
allFeats.drop("index",axis=1,inplace=True)
allFeats

,cIds,Eccentricity,InertialShapeFactor,NPR1,NPR2,PMI1,PMI2,PMI3,RadiusOfGyration,SpherocityIndex,...,SMR_VSA3,SlogP_VSA1,SlogP_VSA2,SlogP_VSA3,EState_VSA1,EState_VSA2,EState_VSA3,VSA_EState1,VSA_EState2,VSA_EState3
0,CIDs00065628,0.993094,0.000967,0.117318,0.929722,960.955541,7615.377756,8191.025810,4.837408,0.107492,...,9.551078,4.899910,45.476431,18.263031,5.969305,6.420822,24.601650,2.022224,17.421914,8.744802
1,CIDs00039562,0.971951,0.000981,0.235183,0.912555,930.166315,3609.231638,3955.082953,3.636559,0.280885,...,9.780485,0.000000,20.856317,6.544756,5.969305,5.693928,21.976247,1.632842,11.272932,15.096254
2,CIDs00216326,0.972349,0.001481,0.233532,0.897611,605.925840,2328.953977,2594.613030,3.265545,0.229019,...,10.216698,11.050456,28.663290,16.133831,11.949021,18.235181,24.216416,0.000000,36.718226,2.265185
3,CIDs00020279,0.959913,0.000857,0.280299,0.759055,885.389572,2397.656981,3158.737923,3.357684,0.129194,...,19.519035,5.733667,48.546905,4.736863,18.435834,17.708331,17.584700,7.156880,11.991406,18.831402
4,CIDs00439693,0.984951,0.001606,0.172832,0.859395,535.080972,2660.652752,3095.961119,3.424367,0.108561,...,9.551078,5.316789,56.568767,4.736863,24.539800,13.151638,17.932612,7.269352,8.147504,31.686702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,CIDs00040926,0.980577,0.001715,0.196136,0.927015,540.477051,2554.499018,2755.617952,3.302148,0.249293,...,9.551078,5.316789,56.568767,4.736863,24.539800,13.151638,17.932612,7.269352,8.147504,31.686702
2571,CIDs00032874,0.935842,0.000178,0.352421,0.713403,4005.316956,8107.929218,11365.144291,4.498942,0.143970,...,0.000000,10.470530,86.739902,20.689085,95.669004,46.831173,0.000000,16.992220,39.739082,53.692132
2572,CIDs23615975,0.932180,0.000797,0.361995,0.751665,942.589265,1957.243771,2603.876175,3.221128,0.235991,...,10.325701,5.316789,75.760591,4.794537,36.991736,6.286161,5.008913,0.000000,31.834661,40.917985
2573,CIDs00016886,0.972479,0.001876,0.232989,0.858752,457.849858,1687.547471,1965.115474,3.001011,0.169517,...,14.535057,11.423411,43.562926,4.736863,24.125577,18.976043,0.000000,6.405907,18.546359,18.391178


1+1

In [ ]:
# steps: make df of properties, cell lines and predictor variable
# wrapper
# and double data first

next: combination descriptors & wrapper for larger dataset

In [8]:
data = pd.read_csv("drugCombDBdata/synergy-score-data/REGRdrugcombs_scored.csv")
print(str(len(data))+" different Combinations")
data.head(2)

498865 different Combinations


,ID,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,HSA
0,1,5-FU,ABT-888,A2058,1.72,6.26,-2.75,5.54
1,2,5-FU,ABT-888,A2058,5.88,12.33,3.33,11.61


In [32]:
len(chemDescr[["drugName","cIds"]])

2575

In [9]:
%%capture --no-display

# generate DF with all features and the drug name as columns
joinFeat = chemDescr[["drugName","cIds"]]
joinFeat.drop_duplicates(subset="cIds",inplace=True)
joinFeat.set_index("cIds",inplace=True)
joinFeat["drugName"] = joinFeat["drugName"].str.lower()
allFeats = allFeats.join(joinFeat,on="cIds",how="left")
allFeats

,cIds,Eccentricity,InertialShapeFactor,NPR1,NPR2,PMI1,PMI2,PMI3,RadiusOfGyration,SpherocityIndex,...,SlogP_VSA1,SlogP_VSA2,SlogP_VSA3,EState_VSA1,EState_VSA2,EState_VSA3,VSA_EState1,VSA_EState2,VSA_EState3,drugName
0,CIDs00065628,0.993094,0.000967,0.117318,0.929722,960.955541,7615.377756,8191.025810,4.837408,0.107492,...,4.899910,45.476431,18.263031,5.969305,6.420822,24.601650,2.022224,17.421914,8.744802,bendamustine
1,CIDs00039562,0.971951,0.000981,0.235183,0.912555,930.166315,3609.231638,3955.082953,3.636559,0.280885,...,0.000000,20.856317,6.544756,5.969305,5.693928,21.976247,1.632842,11.272932,15.096254,lonidamine
2,CIDs00216326,0.972349,0.001481,0.233532,0.897611,605.925840,2328.953977,2594.613030,3.265545,0.229019,...,11.050456,28.663290,16.133831,11.949021,18.235181,24.216416,0.000000,36.718226,2.265185,lenalidomide
3,CIDs00020279,0.959913,0.000857,0.280299,0.759055,885.389572,2397.656981,3158.737923,3.357684,0.129194,...,5.733667,48.546905,4.736863,18.435834,17.708331,17.584700,7.156880,11.991406,18.831402,cladribine
4,CIDs00439693,0.984951,0.001606,0.172832,0.859395,535.080972,2660.652752,3095.961119,3.424367,0.108561,...,5.316789,56.568767,4.736863,24.539800,13.151638,17.932612,7.269352,8.147504,31.686702,pentostatin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2570,CIDs00040926,0.980577,0.001715,0.196136,0.927015,540.477051,2554.499018,2755.617952,3.302148,0.249293,...,5.316789,56.568767,4.736863,24.539800,13.151638,17.932612,7.269352,8.147504,31.686702,co-v
2571,CIDs00032874,0.935842,0.000178,0.352421,0.713403,4005.316956,8107.929218,11365.144291,4.498942,0.143970,...,10.470530,86.739902,20.689085,95.669004,46.831173,0.000000,16.992220,39.739082,53.692132,doxorubicin hydrochloride
2572,CIDs23615975,0.932180,0.000797,0.361995,0.751665,942.589265,1957.243771,2603.876175,3.221128,0.235991,...,5.316789,75.760591,4.794537,36.991736,6.286161,5.008913,0.000000,31.834661,40.917985,nsc 85998
2573,CIDs00016886,0.972479,0.001876,0.232989,0.858752,457.849858,1687.547471,1965.115474,3.001011,0.169517,...,11.423411,43.562926,4.736863,24.125577,18.976043,0.000000,6.405907,18.546359,18.391178,nsc-127716


In [10]:
#read the drug data
data = pd.read_csv("drugCombDBdata/synergy-score-data/REGRdrugcombs_scored.csv")
print(str(len(data))+" different Combinations")
data.drop(["ZIP","Bliss","Loewe"],axis=1, inplace=True)
data["Drug1"] = data["Drug1"].str.lower()
data["Drug2"] = data["Drug2"].str.lower()
data.head(2)

498865 different Combinations


,ID,Drug1,Drug2,Cell line,HSA
0,1,5-fu,abt-888,A2058,5.54
1,2,5-fu,abt-888,A2058,11.61


In [11]:
#double the data
dataReversed = data
dataReversed = dataReversed.rename(columns={"Drug1":"Drug2","Drug2":"Drug1"})
print("number of records we have "+str(len(dataReversed)))
dataReversed.head(2)

number of records we have 498865


,ID,Drug2,Drug1,Cell line,HSA
0,1,5-fu,abt-888,A2058,5.54
1,2,5-fu,abt-888,A2058,11.61


In [12]:
dataDouble = data.append(dataReversed)
dataDouble.drop("ID",axis=1,inplace=True)
dataDouble.reset_index(inplace=True)
dataDouble.drop("index",axis=1,inplace=True)
print(len(dataDouble))
dataDouble.head(2)

997730


,Drug1,Drug2,Cell line,HSA
0,5-fu,abt-888,A2058,5.54
1,5-fu,abt-888,A2058,11.61


In [14]:
with open('./featureSelection/doubledData.pkl', 'wb') as outp:
    pickle.dump(dataDouble, outp)

In [15]:
joinFrame = allFeats.set_index("drugName")
with open('./featureSelection/joinFrame.pkl', 'wb') as outp:
    pickle.dump(joinFrame, outp)
